In [1]:
import sys
sys.path.append('../')
import daq
import file
import globalVAR as Gvar
import visa
import numpy as np
import time

%load_ext autoreload
%autoreload 1

In [2]:
%aimport daq
DAQ = daq.Daq()
DAQ.connect_instr('TDS2024C', 1)
DAQ.connect_instr('KA3005P', 'ttyUSB0')
DAQ.connect_instr('Camera', 17583372)
DAQ.connect_instr('Camera', 17529184)
DAQ.connect_instr('HR4000', 'michaelito')

Oscilloscope ID: TEKTRONIX,TDS 2024C,C046401,CF:91.1CT FV:v24.26

Power supply ID: KORADKA3005PV2.0
Camera ID: Blackfly BFLY-PGE-31S4M,17583372,FV:1.61.3.0
Camera ID: Blackfly BFLY-PGE-50S5M,17529184,FV:1.53.3.0
Spectrometer ID: HR4000,michaelito


In [3]:
start1 = daq.Command(DAQ.instr["17583372"].start_capture, response=False)
start2 = daq.Command(DAQ.instr["17529184"].start_capture, response=False)
pic1 = daq.Command(DAQ.instr["17583372"].retrieve_buffer, response=True)
pic2 = daq.Command(DAQ.instr["17529184"].retrieve_buffer, response=True)
stop1 = daq.Command(DAQ.instr["17583372"].stop_capture, response=False)
stop2 = daq.Command(DAQ.instr["17529184"].stop_capture, response=False)

In [4]:
serial = "None"
v = np.linspace(0, 20, 1000)
for i in range(1000):
    com = daq.Command(DAQ.instr[serial].set_voltage, (v[i],), False)
    DAQ.command_queue[serial].put(com)

In [6]:
dataSet = Gvar.getDataSetNum()
file.add_to_log(dataSet)
file.make_dir_struct('META', dataSet)
file.make_dir_struct('IMAGE', dataSet)
def callback(ret, shot):
    data = {}
    data['meta'] = {'Serial number' : 17583372}
    data['image'] = ret
    file.save_IMAGE(data, dataSet, shot)

DAQ.command_queue["17583372"].put(start1)
for i in range(10):
    com = daq.Command(DAQ.instr["17583372"].retrieve_buffer, callback=callback, callargs=(i,))
    DAQ.command_queue["17583372"].put(com)
DAQ.command_queue["17583372"].put(stop1)

In [6]:
names = []
for name in DAQ.instr:
    names.append(name)
for name in names:
    DAQ.disconnect_instr(name)

An exception was encountered in thread: 17583372, b'Camera has not been connected.'
An exception was encountered in thread: 17583372, b'Camera has not been connected.'
An exception was encountered in thread: 17583372, b'Camera has not been connected.'
An exception was encountered in thread: 17583372, b'Camera has not been connected.'
An exception was encountered in thread: 17583372, b'Camera has not been connected.'
An exception was encountered in thread: 17583372, b'Camera has not been connected.'
An exception was encountered in thread: 17583372, b'Camera has not been connected.'
An exception was encountered in thread: 17583372, b'Camera has not been connected.'